# **Import Library**

In [ ]:
!pip install sastrawi

In [ ]:
!pip install swifter

In [ ]:
!pip install fasttext

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import swifter

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import accuracy_score, precision_score
from sklearn.svm import SVC

from sklearn.preprocessing import LabelEncoder
import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout, GlobalMaxPooling1D

import fasttext
import datetime as dt  # Manipulasi data waktu dan tanggal
import re  # Modul untuk bekerja dengan ekspresi reguler
import string
from nltk.tokenize import word_tokenize  # Tokenisasi teks
from nltk.corpus import stopwords  # Daftar kata-kata berhenti dalam teks

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory  # Stemming (penghilangan imbuhan kata) dalam bahasa Indonesia
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory  # Menghapus kata-kata berhenti dalam bahasa Indonesia

import pickle

In [ ]:
import nltk
nltk.download('punkt')  # Mengunduh dataset yang diperlukan untuk tokenisasi teks.
nltk.download('punkt_tab')
nltk.download('stopwords')  # Mengunduh dataset yang berisi daftar kata-kata berhenti (stop words) dalam berbagai bahasa.

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
pip freeze > requirements.txt

# **Loading Dataset**

In [ ]:
app_reviews_df = pd.read_csv("hasil_scrapping.csv")

In [ ]:
app_reviews_df

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,1c880134-d6f2-43f1-8b5d-c81ecb342fd6,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Update datanya sedikit lambat, tagihan listrik...",5,86,5.2.9,2021-12-21 09:57:14,Hi Kak. Makasih buat penilaiannya. Semoga PLN ...,2021-12-22 07:13:23,5.2.9
1,df908629-4678-4bae-a03c-2ce455e21925,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,aplikasi sangat bagus sangat membantu dalam ma...,5,80,5.2.68,2025-03-26 01:40:28,"Hello Sumarni2, Makasih yah buat bintangnya, s...",2025-03-29 01:37:42,5.2.68
2,2b4ec092-184e-4d46-b400-1e5806d048bd,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Secara Keseluruhan sudah keren, dan sangat mem...",4,3,5.2.68,2025-04-07 21:53:10,"Hai Kak Fahmi, terima kasih buat atensinya ter...",2025-04-08 03:28:04,5.2.68
3,662d6dbe-dc51-4701-82a9-3b42cbfc200f,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,aplikasi sangat berguna dan praktis .memberi k...,5,16,5.2.68,2025-03-23 18:43:33,Terima kasih buat penilaiannya Kak Suhaeni. Mo...,2025-03-27 08:32:13,5.2.68
4,bd59fa08-4e89-42b2-a539-1c06f267009d,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Kenapa setiap/ pada tanggal 22 adalah mulai tg...,2,7,5.2.68,2025-03-23 14:59:38,"Tks utk segala feedback & masukannya, Kak Bona...",2025-03-23 15:06:14,5.2.68
...,...,...,...,...,...,...,...,...,...,...,...
130495,127865ac-2969-41b9-9062-91a749a32c36,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Aplikasi ini sangat membantu sekali,terima kasih",5,0,5.2.13,2022-04-18 04:48:51,"Hay Kak Oji, mimin seneng deh kalo Kakak suka ...",2022-04-18 06:11:13,5.2.13
130496,31d165da-dbce-43da-9e34-25f448c2c54c,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Puas dg pngaduan. Dan teratasi dg puas. Terima...,5,0,5.2.53,2024-06-20 10:36:02,Wak Kak Azhar. Makasih udah suka & puas menggu...,2024-08-04 02:27:05,5.2.53
130497,fb33b3af-5fdf-4e51-9a2f-7c1369e29489,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Pln mobile selalu didepan..mantap,5,0,5.2.30,2022-10-06 06:21:58,"Hallo Kak Ikram, makasih yah buat bintangnya, ...",2022-10-08 02:10:45,5.2.30
130498,d31de6da-82c6-4732-b565-1c5da2ffddb7,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Saya suka aplikasi PLN Mobile karena sangat ng...,5,0,5.2.26,2022-09-08 08:19:13,"Yuhuu Kaka, mimin seneng deh kalo kaka suka sa...",2022-09-08 09:19:40,5.2.26


In [ ]:
app_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130500 entries, 0 to 130499
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   reviewId              130500 non-null  object        
 1   userName              130500 non-null  object        
 2   userImage             130500 non-null  object        
 3   content               130500 non-null  object        
 4   score                 130500 non-null  int64         
 5   thumbsUpCount         130500 non-null  int64         
 6   reviewCreatedVersion  115485 non-null  object        
 7   at                    130500 non-null  datetime64[ns]
 8   replyContent          128926 non-null  object        
 9   repliedAt             128926 non-null  datetime64[ns]
 10  appVersion            115485 non-null  object        
dtypes: datetime64[ns](2), int64(2), object(7)
memory usage: 11.0+ MB


In [ ]:
app_reviews_df.drop_duplicates(inplace = True)

In [ ]:
app_reviews_df.dropna(inplace = True)

In [ ]:
app_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 114094 entries, 0 to 130499
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   reviewId              114094 non-null  object        
 1   userName              114094 non-null  object        
 2   userImage             114094 non-null  object        
 3   content               114094 non-null  object        
 4   score                 114094 non-null  int64         
 5   thumbsUpCount         114094 non-null  int64         
 6   reviewCreatedVersion  114094 non-null  object        
 7   at                    114094 non-null  datetime64[ns]
 8   replyContent          114094 non-null  object        
 9   repliedAt             114094 non-null  datetime64[ns]
 10  appVersion            114094 non-null  object        
dtypes: datetime64[ns](2), int64(2), object(7)
memory usage: 10.4+ MB


# **Preprocessing**

In [ ]:
df = app_reviews_df.sample(n=20000, random_state=42)

In [ ]:
def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # remove mentions
    text = re.sub(r'#[A-Za-z0-9]+', '', text) # remove hashtag
    text = re.sub(r'RT[\s]', '', text) # remove RT
    text = re.sub(r"http\S+", '', text) # remove link
    text = re.sub(r'[0-9]+', '', text) # remove numbers
    text = re.sub(r'[^\w\s]', '', text) # remove punctuation


    text = text.replace('\n', ' ')
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.strip(' ')
    return text

def casefoldingText(text):
    text = text.lower()
    return text

def tokenizingText(text):
    text = word_tokenize(text)
    return text

def filteringText(text):
    listStopwords = set(stopwords.words('indonesian'))
    listStopwords1 = set(stopwords.words('english'))
    listStopwords.update(listStopwords1)
    listStopwords.update(['iya','yaa','gak','nya','na','sih','ku',"di","ga","ya","gaa","loh","kah","woi","woii","woy"])
    filtered = []
    for txt in text:
        if txt not in listStopwords:
            filtered.append(txt)
    text = filtered
    return text


def stemmingText(text):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()

    words = text.split()
    stemmed_words = [stemmer.stem(word) for word in words]
    stemmed_text = ' '.join(stemmed_words)
    print(" row stemmed")

    return stemmed_text

def toSentence(list_words):
    sentence = ' '.join(word for word in list_words)
    return sentence

In [ ]:
slangwords = {"@": "di", "abis": "habis", "wtb": "beli", "masi": "masih", "wts": "jual", "wtt": "tukar", "bgt": "banget", "maks": "maksimal", "plisss": "tolong", "bgttt": "banget", "indo": "indonesia", "bgtt": "banget", "ad": "ada", "rv": "redvelvet", "plis": "tolong", "pls": "tolong", "cr": "sumber", "cod": "bayar ditempat", "adlh": "adalah", "afaik": "as far as i know", "ahaha": "haha", "aj": "saja", "ajep-ajep": "dunia gemerlap", "ak": "saya", "akika": "aku", "akkoh": "aku", "akuwh": "aku", "alay": "norak", "alow": "halo", "ambilin": "ambilkan", "ancur": "hancur", "anjrit": "anjing", "anter": "antar", "ap2": "apa-apa", "apasih": "apa sih", "apes": "sial", "aps": "apa", "aq": "saya", "aquwh": "aku", "asbun": "asal bunyi", "aseekk": "asyik", "asekk": "asyik", "asem": "asam", "aspal": "asli tetapi palsu", "astul": "asal tulis", "ato": "atau", "au ah": "tidak mau tahu", "awak": "saya", "ay": "sayang", "ayank": "sayang", "b4": "sebelum", "bakalan": "akan", "bandes": "bantuan desa", "bangedh": "banget", "banpol": "bantuan polisi", "banpur": "bantuan tempur", "basbang": "basi", "bcanda": "bercanda", "bdg": "bandung", "begajulan": "nakal", "beliin": "belikan", "bencong": "banci", "bentar": "sebentar", "ber3": "bertiga", "beresin": "membereskan", "bete": "bosan", "beud": "banget", "bg": "abang", "bgmn": "bagaimana", "bgt": "banget", "bijimane": "bagaimana", "bintal": "bimbingan mental", "bkl": "akan", "bknnya": "bukannya", "blegug": "bodoh", "blh": "boleh", "bln": "bulan", "blum": "belum", "bnci": "benci", "bnran": "yang benar", "bodor": "lucu", "bokap": "ayah", "boker": "buang air besar", "bokis": "bohong", "boljug": "boleh juga", "bonek": "bocah nekat", "boyeh": "boleh", "br": "baru", "brg": "bareng", "bro": "saudara laki-laki", "bru": "baru", "bs": "bisa", "bsen": "bosan", "bt": "buat", "btw": "ngomong-ngomong", "buaya": "tidak setia", "bubbu": "tidur", "bubu": "tidur", "bumil": "ibu hamil", "bw": "bawa", "bwt": "buat", "byk": "banyak", "byrin": "bayarkan", "cabal": "sabar", "cadas": "keren", "calo": "makelar", "can": "belum", "capcus": "pergi", "caper": "cari perhatian", "ce": "cewek", "cekal": "cegah tangkal", "cemen": "penakut", "cengengesan": "tertawa", "cepet": "cepat", "cew": "cewek", "chuyunk": "sayang", "cimeng": "ganja", "cipika cipiki": "cium pipi kanan cium pipi kiri", "ciyh": "sih", "ckepp": "cakep", "ckp": "cakep", "cmiiw": "correct me if i'm wrong", "cmpur": "campur", "cong": "banci", "conlok": "cinta lokasi", "cowwyy": "maaf", "cp": "siapa", "cpe": "capek", "cppe": "capek", "cucok": "cocok", "cuex": "cuek", "cumi": "Cuma miscall", "cups": "culun", "curanmor": "pencurian kendaraan bermotor", "curcol": "curahan hati colongan", "cwek": "cewek", "cyin": "cinta", "d": "di", "dah": "deh", "dapet": "dapat", "de": "adik", "dek": "adik", "demen": "suka", "deyh": "deh", "dgn": "dengan", "diancurin": "dihancurkan", "dimaafin": "dimaafkan", "dimintak": "diminta", "disono": "di sana", "dket": "dekat", "dkk": "dan kawan-kawan", "dll": "dan lain-lain", "dlu": "dulu", "dngn": "dengan", "dodol": "bodoh", "doku": "uang", "dongs": "dong", "dpt": "dapat", "dri": "dari", "drmn": "darimana", "drtd": "dari tadi", "dst": "dan seterusnya", "dtg": "datang", "duh": "aduh", "duren": "durian", "ed": "edisi", "egp": "emang gue pikirin", "eke": "aku", "elu": "kamu", "emangnya": "memangnya", "emng": "memang", "endak": "tidak", "enggak": "tidak", "envy": "iri", "ex": "mantan", "fax": "facsimile", "fifo": "first in first out", "folbek": "follow back", "fyi": "sebagai informasi", "gaada": "tidak ada uang", "gag": "tidak", "gaje": "tidak jelas", "gak papa": "tidak apa-apa", "gan": "juragan", "gaptek": "gagap teknologi", "gatek": "gagap teknologi", "gawe": "kerja", "gbs": "tidak bisa", "gebetan": "orang yang disuka", "geje": "tidak jelas", "gepeng": "gelandangan dan pengemis", "ghiy": "lagi", "gile": "gila", "gimana": "bagaimana", "gino": "gigi nongol", "githu": "gitu", "gj": "tidak jelas", "gmana": "bagaimana", "gn": "begini", "goblok": "bodoh", "golput": "golongan putih", "gowes": "mengayuh sepeda", "gpny": "tidak punya", "gr": "gede rasa", "gretongan": "gratisan", "gtau": "tidak tahu", "gua": "saya", "guoblok": "goblok", "gw": "saya", "ha": "tertawa", "haha": "tertawa", "hallow": "halo", "hankam": "pertahanan dan keamanan", "hehe": "he", "helo": "halo", "hey": "hai", "hlm": "halaman", "hny": "hanya", "hoax": "isu bohong", "hr": "hari", "hrus": "harus", "hubdar": "perhubungan darat", "huff": "mengeluh", "hum": "rumah", "humz": "rumah", "ilang": "hilang", "ilfil": "tidak suka", "imho": "in my humble opinion", "imoetz": "imut", "item": "hitam", "itungan": "hitungan", "iye": "iya", "ja": "saja", "jadiin": "jadi", "jaim": "jaga image", "jayus": "tidak lucu", "jdi": "jadi", "jem": "jam", "jga": "juga", "jgnkan": "jangankan", "jir": "anjing", "jln": "jalan", "jomblo": "tidak punya pacar", "jubir": "juru bicara", "jutek": "galak", "k": "ke", "kab": "kabupaten", "kabor": "kabur", "kacrut": "kacau", "kadiv": "kepala divisi", "kagak": "tidak", "kalo": "kalau", "kampret": "sialan", "kamtibmas": "keamanan dan ketertiban masyarakat", "kamuwh": "kamu", "kanwil": "kantor wilayah", "karna": "karena", "kasubbag": "kepala subbagian", "katrok": "kampungan", "kayanya": "kayaknya", "kbr": "kabar", "kdu": "harus", "kec": "kecamatan", "kejurnas": "kejuaraan nasional", "kekeuh": "keras kepala", "kel": "kelurahan", "kemaren": "kemarin", "kepengen": "mau", "kepingin": "mau", "kepsek": "kepala sekolah", "kesbang": "kesatuan bangsa", "kesra": "kesejahteraan rakyat", "ketrima": "diterima", "kgiatan": "kegiatan", "kibul": "bohong", "kimpoi": "kawin", "kl": "kalau", "klianz": "kalian", "kloter": "kelompok terbang", "klw": "kalau", "km": "kamu", "kmps": "kampus", "kmrn": "kemarin", "knal": "kenal", "knp": "kenapa", "kodya": "kota madya", "komdis": "komisi disiplin", "komsov": "komunis sovyet", "kongkow": "kumpul bareng teman-teman", "kopdar": "kopi darat", "korup": "korupsi", "kpn": "kapan", "krenz": "keren", "krm": "kirim", "kt": "kita", "ktmu": "ketemu", "ktr": "kantor", "kuper": "kurang pergaulan", "kw": "imitasi", "kyk": "seperti", "la": "lah", "lam": "salam", "lamp": "lampiran", "lanud": "landasan udara", "latgab": "latihan gabungan", "lebay": "berlebihan", "leh": "boleh", "lelet": "lambat", "lemot": "lambat", "lgi": "lagi", "lgsg": "langsung", "liat": "lihat", "litbang": "penelitian dan pengembangan", "lmyn": "lumayan", "lo": "kamu", "loe": "kamu", "lola": "lambat berfikir", "louph": "cinta", "low": "kalau", "lp": "lupa", "luber": "langsung, umum, bebas, dan rahasia", "luchuw": "lucu", "lum": "belum", "luthu": "lucu", "lwn": "lawan", "maacih": "terima kasih", "mabal": "bolos", "macem": "macam", "macih": "masih", "maem": "makan", "magabut": "makan gaji buta", "maho": "homo", "mak jang": "kaget", "maksain": "memaksa", "malem": "malam", "mam": "makan", "maneh": "kamu", "maniez": "manis", "mao": "mau", "masukin": "masukkan", "melu": "ikut", "mepet": "dekat sekali", "mgu": "minggu", "migas": "minyak dan gas bumi", "mikol": "minuman beralkohol", "miras": "minuman keras", "mlah": "malah", "mngkn": "mungkin", "mo": "mau", "mokad": "mati", "moso": "masa", "mpe": "sampai", "msk": "masuk", "mslh": "masalah", "mt": "makan teman", "mubes": "musyawarah besar", "mulu": "melulu", "mumpung": "selagi", "munas": "musyawarah nasional", "muntaber": "muntah dan berak", "musti": "mesti", "muupz": "maaf", "mw": "now watching", "n": "dan", "nanam": "menanam", "nanya": "bertanya", "napa": "kenapa", "napi": "narapidana", "napza": "narkotika, alkohol, psikotropika, dan zat adiktif ", "narkoba": "narkotika, psikotropika, dan obat terlarang", "nasgor": "nasi goreng", "nda": "tidak", "ndiri": "sendiri", "ne": "ini", "nekolin": "neokolonialisme", "nembak": "menyatakan cinta", "ngabuburit": "menunggu berbuka puasa", "ngaku": "mengaku", "ngambil": "mengambil", "nganggur": "tidak punya pekerjaan", "ngapah": "kenapa", "ngaret": "terlambat", "ngasih": "memberikan", "ngebandel": "berbuat bandel", "ngegosip": "bergosip", "ngeklaim": "mengklaim", "ngeksis": "menjadi eksis", "ngeles": "berkilah", "ngelidur": "menggigau", "ngerampok": "merampok", "ngga": "tidak", "ngibul": "berbohong", "ngiler": "mau", "ngiri": "iri", "ngisiin": "mengisikan", "ngmng": "bicara", "ngomong": "bicara", "ngubek2": "mencari-cari", "ngurus": "mengurus", "nie": "ini", "nih": "ini", "niyh": "nih", "nmr": "nomor", "nntn": "nonton", "nobar": "nonton bareng", "np": "now playing", "ntar": "nanti", "ntn": "nonton", "numpuk": "bertumpuk", "nutupin": "menutupi", "nyari": "mencari", "nyekar": "menyekar", "nyicil": "mencicil", "nyoblos": "mencoblos", "nyokap": "ibu", "ogah": "tidak mau", "ol": "online", "ongkir": "ongkos kirim", "oot": "out of topic", "org2": "orang-orang", "ortu": "orang tua", "otda": "otonomi daerah", "otw": "on the way, sedang di jalan", "pacal": "pacar", "pake": "pakai", "pala": "kepala", "pansus": "panitia khusus", "parpol": "partai politik", "pasutri": "pasangan suami istri", "pd": "pada", "pede": "percaya diri", "pelatnas": "pemusatan latihan nasional", "pemda": "pemerintah daerah", "pemkot": "pemerintah kota", "pemred": "pemimpin redaksi", "penjas": "pendidikan jasmani", "perda": "peraturan daerah", "perhatiin": "perhatikan", "pesenan": "pesanan", "pgang": "pegang", "pi": "tapi", "pilkada": "pemilihan kepala daerah", "pisan": "sangat", "pk": "penjahat kelamin", "plg": "paling", "pmrnth": "pemerintah", "polantas": "polisi lalu lintas", "ponpes": "pondok pesantren", "pp": "pulang pergi", "prg": "pergi", "prnh": "pernah", "psen": "pesan", "pst": "pasti", "pswt": "pesawat", "pw": "posisi nyaman", "qmu": "kamu", "rakor": "rapat koordinasi", "ranmor": "kendaraan bermotor", "re": "reply", "ref": "referensi", "rehab": "rehabilitasi", "rempong": "sulit", "repp": "balas", "restik": "reserse narkotika", "rhs": "rahasia", "rmh": "rumah", "ru": "baru", "ruko": "rumah toko", "rusunawa": "rumah susun sewa", "ruz": "terus", "saia": "saya", "salting": "salah tingkah", "sampe": "sampai", "samsek": "sama sekali", "sapose": "siapa", "satpam": "satuan pengamanan", "sbb": "sebagai berikut", "sbh": "sebuah", "sbnrny": "sebenarnya", "scr": "secara", "sdgkn": "sedangkan", "sdkt": "sedikit", "se7": "setuju", "sebelas dua belas": "mirip", "sembako": "sembilan bahan pokok", "sempet": "sempat", "sendratari": "seni drama tari", "sgt": "sangat", "shg": "sehingga", "siech": "sih", "sikon": "situasi dan kondisi", "sinetron": "sinema elektronik", "siramin": "siramkan", "sj": "saja", "skalian": "sekalian", "sklh": "sekolah", "skt": "sakit", "slesai": "selesai", "sll": "selalu", "slma": "selama", "slsai": "selesai", "smpt": "sempat", "smw": "semua", "sndiri": "sendiri", "soljum": "sholat jumat", "songong": "sombong", "sory": "maaf", "sosek": "sosial-ekonomi", "sotoy": "sok tahu", "spa": "siapa", "sppa": "siapa", "spt": "seperti", "srtfkt": "sertifikat", "stiap": "setiap", "stlh": "setelah", "suk": "masuk", "sumpek": "sempit", "syg": "sayang", "t4": "tempat", "tajir": "kaya", "tau": "tahu", "taw": "tahu", "td": "tadi", "tdk": "tidak", "teh": "kakak perempuan", "telat": "terlambat", "telmi": "telat berpikir", "temen": "teman", "tengil": "menyebalkan", "tepar": "terkapar", "tggu": "tunggu", "tgu": "tunggu", "thankz": "terima kasih", "thn": "tahun", "tilang": "bukti pelanggaran", "tipiwan": "TvOne", "tks": "terima kasih", "tlp": "telepon", "tls": "tulis", "tmbah": "tambah", "tmen2": "teman-teman", "tmpah": "tumpah", "tmpt": "tempat", "tngu": "tunggu", "tnyta": "ternyata", "tokai": "tai", "toserba": "toko serba ada", "tpi": "tapi", "trdhulu": "terdahulu", "trima": "terima kasih", "trm": "terima", "trs": "terus", "trutama": "terutama", "ts": "penulis", "tst": "tahu sama tahu", "ttg": "tentang", "tuch": "tuh", "tuir": "tua", "tw": "tahu", "u": "kamu", "ud": "sudah", "udah": "sudah", "ujg": "ujung", "ul": "ulangan", "unyu": "lucu", "uplot": "unggah", "urang": "saya", "usah": "perlu", "utk": "untuk", "valas": "valuta asing", "w/": "dengan", "wadir": "wakil direktur", "wamil": "wajib militer", "warkop": "warung kopi", "warteg": "warung tegal", "wat": "buat", "wkt": "waktu", "wtf": "what the fuck", "xixixi": "tertawa", "ya": "iya", "yap": "iya", "yaudah": "ya sudah", "yawdah": "ya sudah", "yg": "yang", "yl": "yang lain", "yo": "iya", "yowes": "ya sudah", "yup": "iya", "7an": "tujuan", "ababil": "abg labil", "acc": "accord", "adlah": "adalah", "adoh": "aduh", "aha": "tertawa", "aing": "saya", "aja": "saja", "ajj": "saja", "aka": "dikenal juga sebagai", "akko": "aku", "akku": "aku", "akyu": "aku", "aljasa": "asal jadi saja", "ama": "sama", "ambl": "ambil", "anjir": "anjing", "ank": "anak", "ap": "apa", "apaan": "apa", "ape": "apa", "aplot": "unggah", "apva": "apa", "aqu": "aku", "asap": "sesegera mungkin", "aseek": "asyik", "asek": "asyik", "aseknya": "asyiknya", "asoy": "asyik", "astrojim": "astagfirullahaladzim", "ath": "kalau begitu", "atuh": "kalau begitu", "ava": "avatar", "aws": "awas", "ayang": "sayang", "ayok": "ayo", "bacot": "banyak bicara", "bales": "balas", "bangdes": "pembangunan desa", "bangkotan": "tua", "banpres": "bantuan presiden", "bansarkas": "bantuan sarana kesehatan", "bazis": "badan amal, zakat, infak, dan sedekah", "bcoz": "karena", "beb": "sayang", "bejibun": "banyak", "belom": "belum", "bener": "benar", "ber2": "berdua", "berdikari": "berdiri di atas kaki sendiri", "bet": "banget", "beti": "beda tipis", "beut": "banget", "bgd": "banget", "bgs": "bagus", "bhubu": "tidur", "bimbuluh": "bimbingan dan penyuluhan", "bisi": "kalau-kalau", "bkn": "bukan", "bl": "beli", "blg": "bilang", "blm": "belum", "bls": "balas", "bnchi": "benci", "bngung": "bingung", "bnyk": "banyak", "bohay": "badan aduhai", "bokep": "porno", "bokin": "pacar", "bole": "boleh", "bolot": "bodoh", "bonyok": "ayah ibu", "bpk": "bapak", "brb": "segera kembali", "brngkt": "berangkat", "brp": "berapa", "brur": "saudara laki-laki", "bsa": "bisa", "bsk": "besok", "bu_bu": "tidur", "bubarin": "bubarkan", "buber": "buka bersama", "bujubune": "luar biasa", "buser": "buru sergap", "bwhn": "bawahan", "byar": "bayar", "byr": "bayar", "c8": "chat", "cabut": "pergi", "caem": "cakep", "cama-cama": "sama-sama", "cangcut": "celana dalam", "cape": "capek", "caur": "jelek", "cekak": "tidak ada uang", "cekidot": "coba lihat", "cemplungin": "cemplungkan", "ceper": "pendek", "ceu": "kakak perempuan", "cewe": "cewek", "cibuk": "sibuk", "cin": "cinta", "ciye": "cie", "ckck": "ck", "clbk": "cinta lama bersemi kembali", "cmpr": "campur", "cnenk": "senang", "congor": "mulut", "cow": "cowok", "coz": "karena", "cpa": "siapa", "gokil": "gila", "gombal": "suka merayu", "gpl": "tidak pakai lama", "gpp": "tidak apa-apa", "gretong": "gratis", "gt": "begitu", "gtw": "tidak tahu", "gue": "saya", "guys": "teman-teman", "gws": "cepat sembuh", "haghaghag": "tertawa", "hakhak": "tertawa", "handak": "bahan peledak", "hansip": "pertahanan sipil", "hellow": "halo", "helow": "halo", "hi": "hai", "hlng": "hilang", "hnya": "hanya", "houm": "rumah", "hrs": "harus", "hubad": "hubungan angkatan darat", "hubla": "perhubungan laut", "huft": "mengeluh", "humas": "hubungan masyarakat", "idk": "saya tidak tahu", "ilfeel": "tidak suka", "imba": "jago sekali", "imoet": "imut", "info": "informasi", "itung": "hitung", "isengin": "bercanda", "iyala": "iya lah", "iyo": "iya", "jablay": "jarang dibelai", "jadul": "jaman dulu", "jancuk": "anjing", "jd": "jadi", "jdikan": "jadikan", "jg": "juga", "jgn": "jangan", "jijay": "jijik", "jkt": "jakarta", "jnj": "janji", "jth": "jatuh", "jurdil": "jujur adil", "jwb": "jawab", "ka": "kakak", "kabag": "kepala bagian", "kacian": "kasihan", "kadit": "kepala direktorat", "kaga": "tidak", "kaka": "kakak", "kamtib": "keamanan dan ketertiban", "kamuh": "kamu", "kamyu": "kamu", "kapt": "kapten", "kasat": "kepala satuan", "kasubbid": "kepala subbidang", "kau": "kamu", "kbar": "kabar", "kcian": "kasihan", "keburu": "terlanjur", "kedubes": "kedutaan besar", "kek": "seperti", "keknya": "kayaknya", "keliatan": "kelihatan", "keneh": "masih", "kepikiran": "terpikirkan", "kepo": "mau tahu urusan orang", "kere": "tidak punya uang", "kesian": "kasihan", "ketauan": "ketahuan", "keukeuh": "keras kepala", "khan": "kan", "kibus": "kaki busuk", "kk": "kakak", "klian": "kalian", "klo": "kalau", "kluarga": "keluarga", "klwrga": "keluarga", "kmari": "kemari", "kmpus": "kampus", "kn": "kan", "knl": "kenal", "knpa": "kenapa", "kog": "kok", "kompi": "komputer", "komtiong": "komunis Tiongkok", "konjen": "konsulat jenderal", "koq": "kok", "kpd": "kepada", "kptsan": "keputusan", "krik": "garing", "krn": "karena", "ktauan": "ketahuan", "ktny": "katanya", "kudu": "harus", "kuq": "kok", "ky": "seperti", "kykny": "kayanya", "laka": "kecelakaan", "lambreta": "lambat", "lansia": "lanjut usia", "lapas": "lembaga pemasyarakatan", "lbur": "libur", "lekong": "laki-laki", "lg": "lagi", "lgkp": "lengkap", "lht": "lihat", "linmas": "perlindungan masyarakat", "lmyan": "lumayan", "lngkp": "lengkap", "loch": "loh", "lol": "tertawa", "lom": "belum", "loupz": "cinta", "lowh": "kamu", "lu": "kamu", "luchu": "lucu", "luff": "cinta", "luph": "cinta", "lw": "kamu", "lwt": "lewat", "maaciw": "terima kasih", "mabes": "markas besar", "macem-macem": "macam-macam", "madesu": "masa depan suram", "maen": "main", "mahatma": "maju sehat bersama", "mak": "ibu", "makasih": "terima kasih", "malah": "bahkan", "malu2in": "memalukan", "mamz": "makan", "manies": "manis", "mantep": "mantap", "markus": "makelar kasus", "mba": "mbak", "mending": "lebih baik", "mgkn": "mungkin", "mhn": "mohon", "miker": "minuman keras", "milis": "mailing list", "mksd": "maksud", "mls": "malas", "mnt": "minta", "moge": "motor gede", "mokat": "mati", "mosok": "masa", "msh": "masih", "mskpn": "meskipun", "msng2": "masing-masing", "muahal": "mahal", "muker": "musyawarah kerja", "mumet": "pusing", "muna": "munafik", "munaslub": "musyawarah nasional luar biasa", "musda": "musyawarah daerah", "muup": "maaf", "muuv": "maaf", "nal": "kenal", "nangis": "menangis", "naon": "apa", "napol": "narapidana politik", "naq": "anak", "narsis": "bangga pada diri sendiri", "nax": "anak", "ndak": "tidak", "ndut": "gendut", "nekolim": "neokolonialisme", "nelfon": "menelepon", "ngabis2in": "menghabiskan", "ngakak": "tertawa", "ngambek": "marah", "ngampus": "pergi ke kampus", "ngantri": "mengantri", "ngapain": "sedang apa", "ngaruh": "berpengaruh", "ngawur": "berbicara sembarangan", "ngeceng": "kumpul bareng-bareng", "ngeh": "sadar", "ngekos": "tinggal di kos", "ngelamar": "melamar", "ngeliat": "melihat", "ngemeng": "bicara terus-terusan", "ngerti": "mengerti", "nggak": "tidak", "ngikut": "ikut", "nginep": "menginap", "ngisi": "mengisi", "ngmg": "bicara", "ngocol": "lucu", "ngomongin": "membicarakan", "ngumpul": "berkumpul", "ni": "ini", "nyasar": "tersesat", "nyariin": "mencari", "nyiapin": "mempersiapkan", "nyiram": "menyiram", "nyok": "ayo", "o/": "oleh", "ok": "ok", "priksa": "periksa", "pro": "profesional", "psn": "pesan", "psti": "pasti", "puanas": "panas", "qmo": "kamu", "qt": "kita", "rame": "ramai", "raskin": "rakyat miskin", "red": "redaksi", "reg": "register", "rejeki": "rezeki", "renstra": "rencana strategis", "reskrim": "reserse kriminal", "sni": "sini", "somse": "sombong sekali", "sorry": "maaf", "sosbud": "sosial-budaya", "sospol": "sosial-politik", "sowry": "maaf", "spd": "sepeda", "sprti": "seperti", "spy": "supaya", "stelah": "setelah", "subbag": "subbagian", "sumbangin": "sumbangkan", "sy": "saya", "syp": "siapa", "tabanas": "tabungan pembangunan nasional", "tar": "nanti", "taun": "tahun", "tawh": "tahu", "tdi": "tadi", "te2p": "tetap", "tekor": "rugi", "telkom": "telekomunikasi", "telp": "telepon", "temen2": "teman-teman", "tengok": "menjenguk", "terbitin": "terbitkan", "tgl": "tanggal", "thanks": "terima kasih", "thd": "terhadap", "thx": "terima kasih", "tipi": "TV", "tkg": "tukang", "tll": "terlalu", "tlpn": "telepon", "tman": "teman", "tmbh": "tambah", "tmn2": "teman-teman", "tmph": "tumpah", "tnda": "tanda", "tnh": "tanah", "togel": "toto gelap", "tp": "tapi", "tq": "terima kasih", "trgntg": "tergantung", "trims": "terima kasih", "cb": "coba", "y": "ya", "munfik": "munafik", "reklamuk": "reklamasi", "sma": "sama", "tren": "trend", "ngehe": "kesal", "mz": "mas", "analisise": "analisis", "sadaar": "sadar", "sept": "september", "nmenarik": "menarik", "zonk": "bodoh", "rights": "benar", "simiskin": "miskin", "ngumpet": "sembunyi", "hardcore": "keras", "akhirx": "akhirnya", "solve": "solusi", "watuk": "batuk", "ngebully": "intimidasi", "masy": "masyarakat", "still": "masih", "tauk": "tahu", "mbual": "bual", "tioghoa": "tionghoa", "ngentotin": "senggama", "kentot": "senggama", "faktakta": "fakta", "sohib": "teman", "rubahnn": "rubah", "trlalu": "terlalu", "nyela": "cela", "heters": "pembenci", "nyembah": "sembah", "most": "paling", "ikon": "lambang", "light": "terang", "pndukung": "pendukung", "setting": "atur", "seting": "akting", "next": "lanjut", "waspadalah": "waspada", "gantengsaya": "ganteng", "parte": "partai", "nyerang": "serang", "nipu": "tipu", "ktipu": "tipu", "jentelmen": "berani", "buangbuang": "buang", "tsangka": "tersangka", "kurng": "kurang", "ista": "nista", "less": "kurang", "koar": "teriak", "paranoid": "takut", "problem": "masalah", "tahi": "kotoran", "tirani": "tiran", "tilep": "tilap", "happy": "bahagia", "tak": "tidak", "penertiban": "tertib", "uasai": "kuasa", "mnolak": "tolak", "trending": "trend", "taik": "tahi", "wkwkkw": "tertawa", "ahokncc": "ahok", "istaa": "nista", "benarjujur": "jujur", "mgkin": "mungkin"}

def fix_slangwords(text):
    words = text.split()
    fixed_words = []

    for word in words:
        if word.lower() in slangwords:
            fixed_words.append(slangwords[word.lower()])
        else:
            fixed_words.append(word)

    fixed_text = ' '.join(fixed_words)
    return fixed_text

In [ ]:
def complete_clean(text):
    text = casefoldingText(text)
    text = fix_slangwords(text)
    text = cleaningText(text)
    text = stemmingText(text)
    text = tokenizingText(text)
    text = filteringText(text)
    return text

In [ ]:
text = "aplikasi sangat bagus dan bermanfaat"
text = complete_clean(text)
print(text)

 row stemmed
['aplikasi', 'bagus', 'manfaat']


In [ ]:
df.shape

(20000, 11)

In [ ]:
df

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
74130,92365fc4-30e9-4471-9d41-fa6cf6ffcc52,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Aplikasinya mantap banget muda dan tercepat,5,0,5.2.26,2022-09-10 00:16:18,"Hallo kak Anis, Makasih yaa buat review & rati...",2022-09-10 02:36:58,5.2.26
1845,9d0e4dc0-4d8b-4e0a-a769-88d2c4343eac,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Selama menggunakan pln mobile transaksi beli t...,5,0,5.2.68,2025-04-09 09:07:45,"Hai Kak Muhip, Tengkyu ya Kak penilaiannya bua...",2025-04-10 10:14:03,5.2.68
9811,3772ca42-1af0-4224-ae68-34e83e8ed84a,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,sangat membantu... saya jadi tidak bingung lag...,5,0,5.2.68,2025-03-12 05:26:23,"Hai Kak Deanti, makasih udah puas gunakan PLN ...",2025-03-25 03:22:48,5.2.68
28797,0f058f79-d65c-4f4b-a95b-879d81b5c9a4,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Aplikasi yang sangat bagus dan mudah digunakan,5,0,5.2.68,2025-03-20 02:59:14,Hai Kak Nasrullah. Mimin seneng deh kalo Kakak...,2025-03-12 22:55:23,5.2.68
2061,0d6eed0d-8e44-4216-90f0-d3da314ed269,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Aplikasi ini sangat membantu sekali dalam hal ...,5,0,5.2.68,2025-03-18 01:27:52,Yuhu Kak Anselma. Waaah banget nih penilaian s...,2025-03-20 07:13:34,5.2.68
...,...,...,...,...,...,...,...,...,...,...,...
79152,08fb0c9c-ca68-4197-9d37-33cfe8516f51,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Bagus pelayanannya.cepat tanggap,sangat puas 🥰",5,0,5.2.57,2024-07-29 06:53:50,Hi Kakak Findo. Makasih udah suka & puas mengg...,2024-07-31 17:13:31,5.2.57
105997,2505c6e6-677d-4127-aad5-162b0b9d5b0c,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Pln mobile mantap,5,0,5.2.66,2025-01-16 04:36:25,"Tks Kak penilaiannya, semoga semakin setia men...",2025-04-06 02:58:22,5.2.66
117672,bd65d976-7b83-4f77-bf57-38c7a3ce9dc9,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,pln mobile luar biasa,5,0,5.2.42,2023-06-14 03:50:55,Helo Kak Yoan. Bisa aja nih Kak. Btw makasih y...,2023-06-18 23:31:06,5.2.42
10062,fdd0369c-cb50-41c2-9402-adae5ff5b3ba,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Pln semakin transparan dengan ada nya aplikasi...,5,0,5.2.53,2025-01-10 09:51:14,Halo Kak Jalaludin. Wah seneng deh baca ulasan...,2025-04-09 04:03:06,5.2.53


In [ ]:
df['text_stopword'] = ''

In [ ]:
for i in range(0, len(df), 100):
  print("Iterasi ke", i)
  df['text_stopword'][i:i+100]= df['content'][i:i+100].swifter.apply(complete_clean)

Iterasi ke 0


Pandas Apply:   0%|          | 0/100 [00:00<?, ?it/s]

 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed

<ipython-input-45-e18f9b8c57fc>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text_stopword'][i:i+100]= df['content'][i:i+100].swifter.apply(complete_clean)


Pandas Apply:   0%|          | 0/100 [00:00<?, ?it/s]

...

 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed

<ipython-input-45-e18f9b8c57fc>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text_stopword'][i:i+100]= df['content'][i:i+100].swifter.apply(complete_clean)


Pandas Apply:   0%|          | 0/100 [00:00<?, ?it/s]

 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed
 row stemmed

<ipython-input-45-e18f9b8c57fc>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text_stopword'][i:i+100]= df['content'][i:i+100].swifter.apply(complete_clean)


In [ ]:
df['text_akhir']= df['text_stopword'].swifter.apply(toSentence)

Pandas Apply:   0%|          | 0/20000 [00:00<?, ?it/s]

In [ ]:
df

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,text_stopword,text_akhir
74130,92365fc4-30e9-4471-9d41-fa6cf6ffcc52,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Aplikasinya mantap banget muda dan tercepat,5,0,5.2.26,2022-09-10 00:16:18,"Hallo kak Anis, Makasih yaa buat review & rati...",2022-09-10 02:36:58,5.2.26,"[aplikasi, mantap, banget, muda, cepat]",aplikasi mantap banget muda cepat
1845,9d0e4dc0-4d8b-4e0a-a769-88d2c4343eac,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Selama menggunakan pln mobile transaksi beli t...,5,0,5.2.68,2025-04-09 09:07:45,"Hai Kak Muhip, Tengkyu ya Kak penilaiannya bua...",2025-04-10 10:14:03,5.2.68,"[pln, mobile, transaksi, beli, token, mudah]",pln mobile transaksi beli token mudah
9811,3772ca42-1af0-4224-ae68-34e83e8ed84a,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,sangat membantu... saya jadi tidak bingung lag...,5,0,5.2.68,2025-03-12 05:26:23,"Hai Kak Deanti, makasih udah puas gunakan PLN ...",2025-03-25 03:22:48,5.2.68,"[bantu, bingung, lapor, rusa, listrik, rumah, ...",bantu bingung lapor rusa listrik rumah terimak...
28797,0f058f79-d65c-4f4b-a95b-879d81b5c9a4,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Aplikasi yang sangat bagus dan mudah digunakan,5,0,5.2.68,2025-03-20 02:59:14,Hai Kak Nasrullah. Mimin seneng deh kalo Kakak...,2025-03-12 22:55:23,5.2.68,"[aplikasi, bagus, mudah]",aplikasi bagus mudah
2061,0d6eed0d-8e44-4216-90f0-d3da314ed269,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Aplikasi ini sangat membantu sekali dalam hal ...,5,0,5.2.68,2025-03-18 01:27:52,Yuhu Kak Anselma. Waaah banget nih penilaian s...,2025-03-20 07:13:34,5.2.68,"[aplikasi, bantu, ganggu, listrik, lainmantap]",aplikasi bantu ganggu listrik lainmantap
...,...,...,...,...,...,...,...,...,...,...,...,...,...
79152,08fb0c9c-ca68-4197-9d37-33cfe8516f51,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Bagus pelayanannya.cepat tanggap,sangat puas 🥰",5,0,5.2.57,2024-07-29 06:53:50,Hi Kakak Findo. Makasih udah suka & puas mengg...,2024-07-31 17:13:31,5.2.57,"[bagus, pelayanannyacepat, tanggapsangat, puas]",bagus pelayanannyacepat tanggapsangat puas
105997,2505c6e6-677d-4127-aad5-162b0b9d5b0c,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Pln mobile mantap,5,0,5.2.66,2025-01-16 04:36:25,"Tks Kak penilaiannya, semoga semakin setia men...",2025-04-06 02:58:22,5.2.66,"[pln, mobile, mantap]",pln mobile mantap
117672,bd65d976-7b83-4f77-bf57-38c7a3ce9dc9,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,pln mobile luar biasa,5,0,5.2.42,2023-06-14 03:50:55,Helo Kak Yoan. Bisa aja nih Kak. Btw makasih y...,2023-06-18 23:31:06,5.2.42,"[pln, mobile]",pln mobile
10062,fdd0369c-cb50-41c2-9402-adae5ff5b3ba,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Pln semakin transparan dengan ada nya aplikasi...,5,0,5.2.53,2025-01-10 09:51:14,Halo Kak Jalaludin. Wah seneng deh baca ulasan...,2025-04-09 04:03:06,5.2.53,"[pln, transparan, aplikasi, pln, mobile, dafta...",pln transparan aplikasi pln mobile daftar laku...


# **Labelling**

In [ ]:
import csv
import requests
from io import StringIO

lexicon_positive = dict()
response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_positive.csv')


if response.status_code == 200:
    reader = csv.reader(StringIO(response.text), delimiter=',')
    for row in reader:
        lexicon_positive[row[0]] = int(row[1])
else:
    print("Failed to fetch data")


lexicon_negative = dict()
response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_negative.csv')

if response.status_code == 200:
    reader = csv.reader(StringIO(response.text), delimiter=',')
    for row in reader:
        lexicon_negative[row[0]] = int(row[1])
else:
    print("Failed to fetch data")


In [ ]:
def sentiment_analysis_lexicon_indonesia(text):
    score = 0

    for word in text:
        if (word in lexicon_positive):
            score = score + lexicon_positive[word]

    for word in text:
        if (word in lexicon_negative):
            score = score + lexicon_negative[word]

    polarity=''

    if (score > 0):
        polarity = 'positive'
    elif (score < 0):
        polarity = 'negative'
    else:
        polarity = 'neutral'

    return score, polarity

In [ ]:
results = df['text_stopword'].apply(sentiment_analysis_lexicon_indonesia)
results = list(zip(*results))
df['polarity_score'] = results[0]
df['polarity'] = results[1]

print(df['polarity'].value_counts())

polarity
positive    9572
negative    8478
neutral     1950
Name: count, dtype: int64


In [ ]:
df.to_csv('labelled_dataset.csv', index=False)

In [ ]:
df = pd.read_csv('labelled_dataset.csv')

In [ ]:
df

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,text_stopword,text_akhir,polarity_score,polarity
0,92365fc4-30e9-4471-9d41-fa6cf6ffcc52,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Aplikasinya mantap banget muda dan tercepat,5,0,5.2.26,2022-09-10 00:16:18,"Hallo kak Anis, Makasih yaa buat review & rati...",2022-09-10 02:36:58,5.2.26,"['aplikasi', 'mantap', 'banget', 'muda', 'cepat']",aplikasi mantap banget muda cepat,0,neutral
1,9d0e4dc0-4d8b-4e0a-a769-88d2c4343eac,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Selama menggunakan pln mobile transaksi beli t...,5,0,5.2.68,2025-04-09 09:07:45,"Hai Kak Muhip, Tengkyu ya Kak penilaiannya bua...",2025-04-10 10:14:03,5.2.68,"['pln', 'mobile', 'transaksi', 'beli', 'token'...",pln mobile transaksi beli token mudah,6,positive
2,3772ca42-1af0-4224-ae68-34e83e8ed84a,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,sangat membantu... saya jadi tidak bingung lag...,5,0,5.2.68,2025-03-12 05:26:23,"Hai Kak Deanti, makasih udah puas gunakan PLN ...",2025-03-25 03:22:48,5.2.68,"['bantu', 'bingung', 'lapor', 'rusa', 'listrik...",bantu bingung lapor rusa listrik rumah terimak...,9,positive
3,0f058f79-d65c-4f4b-a95b-879d81b5c9a4,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Aplikasi yang sangat bagus dan mudah digunakan,5,0,5.2.68,2025-03-20 02:59:14,Hai Kak Nasrullah. Mimin seneng deh kalo Kakak...,2025-03-12 22:55:23,5.2.68,"['aplikasi', 'bagus', 'mudah']",aplikasi bagus mudah,-3,negative
4,0d6eed0d-8e44-4216-90f0-d3da314ed269,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Aplikasi ini sangat membantu sekali dalam hal ...,5,0,5.2.68,2025-03-18 01:27:52,Yuhu Kak Anselma. Waaah banget nih penilaian s...,2025-03-20 07:13:34,5.2.68,"['aplikasi', 'bantu', 'ganggu', 'listrik', 'la...",aplikasi bantu ganggu listrik lainmantap,-9,negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,08fb0c9c-ca68-4197-9d37-33cfe8516f51,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Bagus pelayanannya.cepat tanggap,sangat puas 🥰",5,0,5.2.57,2024-07-29 06:53:50,Hi Kakak Findo. Makasih udah suka & puas mengg...,2024-07-31 17:13:31,5.2.57,"['bagus', 'pelayanannyacepat', 'tanggapsangat'...",bagus pelayanannyacepat tanggapsangat puas,1,positive
19996,2505c6e6-677d-4127-aad5-162b0b9d5b0c,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Pln mobile mantap,5,0,5.2.66,2025-01-16 04:36:25,"Tks Kak penilaiannya, semoga semakin setia men...",2025-04-06 02:58:22,5.2.66,"['pln', 'mobile', 'mantap']",pln mobile mantap,5,positive
19997,bd65d976-7b83-4f77-bf57-38c7a3ce9dc9,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,pln mobile luar biasa,5,0,5.2.42,2023-06-14 03:50:55,Helo Kak Yoan. Bisa aja nih Kak. Btw makasih y...,2023-06-18 23:31:06,5.2.42,"['pln', 'mobile']",pln mobile,0,neutral
19998,fdd0369c-cb50-41c2-9402-adae5ff5b3ba,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Pln semakin transparan dengan ada nya aplikasi...,5,0,5.2.53,2025-01-10 09:51:14,Halo Kak Jalaludin. Wah seneng deh baca ulasan...,2025-04-09 04:03:06,5.2.53,"['pln', 'transparan', 'aplikasi', 'pln', 'mobi...",pln transparan aplikasi pln mobile daftar laku...,1,positive


# **Pembangunan Model RNN | Extraction Feature : Embedding**

disini saya akan menggunakan model RNN LSTM dengan ektrasi fitur Word Embedding, serta melakukan pembagian data 80/20

In [ ]:
encoder = LabelEncoder()
df['polarity'] = encoder.fit_transform(df['polarity'])

df['text_akhir'] = df['text_akhir'].fillna('').astype(str)

In [ ]:
df

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,text_stopword,text_akhir,polarity_score,polarity
0,92365fc4-30e9-4471-9d41-fa6cf6ffcc52,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Aplikasinya mantap banget muda dan tercepat,5,0,5.2.26,2022-09-10 00:16:18,"Hallo kak Anis, Makasih yaa buat review & rati...",2022-09-10 02:36:58,5.2.26,"['aplikasi', 'mantap', 'banget', 'muda', 'cepat']",aplikasi mantap banget muda cepat,0,1
1,9d0e4dc0-4d8b-4e0a-a769-88d2c4343eac,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Selama menggunakan pln mobile transaksi beli t...,5,0,5.2.68,2025-04-09 09:07:45,"Hai Kak Muhip, Tengkyu ya Kak penilaiannya bua...",2025-04-10 10:14:03,5.2.68,"['pln', 'mobile', 'transaksi', 'beli', 'token'...",pln mobile transaksi beli token mudah,6,2
2,3772ca42-1af0-4224-ae68-34e83e8ed84a,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,sangat membantu... saya jadi tidak bingung lag...,5,0,5.2.68,2025-03-12 05:26:23,"Hai Kak Deanti, makasih udah puas gunakan PLN ...",2025-03-25 03:22:48,5.2.68,"['bantu', 'bingung', 'lapor', 'rusa', 'listrik...",bantu bingung lapor rusa listrik rumah terimak...,9,2
3,0f058f79-d65c-4f4b-a95b-879d81b5c9a4,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Aplikasi yang sangat bagus dan mudah digunakan,5,0,5.2.68,2025-03-20 02:59:14,Hai Kak Nasrullah. Mimin seneng deh kalo Kakak...,2025-03-12 22:55:23,5.2.68,"['aplikasi', 'bagus', 'mudah']",aplikasi bagus mudah,-3,0
4,0d6eed0d-8e44-4216-90f0-d3da314ed269,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Aplikasi ini sangat membantu sekali dalam hal ...,5,0,5.2.68,2025-03-18 01:27:52,Yuhu Kak Anselma. Waaah banget nih penilaian s...,2025-03-20 07:13:34,5.2.68,"['aplikasi', 'bantu', 'ganggu', 'listrik', 'la...",aplikasi bantu ganggu listrik lainmantap,-9,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,08fb0c9c-ca68-4197-9d37-33cfe8516f51,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Bagus pelayanannya.cepat tanggap,sangat puas 🥰",5,0,5.2.57,2024-07-29 06:53:50,Hi Kakak Findo. Makasih udah suka & puas mengg...,2024-07-31 17:13:31,5.2.57,"['bagus', 'pelayanannyacepat', 'tanggapsangat'...",bagus pelayanannyacepat tanggapsangat puas,1,2
19996,2505c6e6-677d-4127-aad5-162b0b9d5b0c,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Pln mobile mantap,5,0,5.2.66,2025-01-16 04:36:25,"Tks Kak penilaiannya, semoga semakin setia men...",2025-04-06 02:58:22,5.2.66,"['pln', 'mobile', 'mantap']",pln mobile mantap,5,2
19997,bd65d976-7b83-4f77-bf57-38c7a3ce9dc9,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,pln mobile luar biasa,5,0,5.2.42,2023-06-14 03:50:55,Helo Kak Yoan. Bisa aja nih Kak. Btw makasih y...,2023-06-18 23:31:06,5.2.42,"['pln', 'mobile']",pln mobile,0,1
19998,fdd0369c-cb50-41c2-9402-adae5ff5b3ba,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Pln semakin transparan dengan ada nya aplikasi...,5,0,5.2.53,2025-01-10 09:51:14,Halo Kak Jalaludin. Wah seneng deh baca ulasan...,2025-04-09 04:03:06,5.2.53,"['pln', 'transparan', 'aplikasi', 'pln', 'mobi...",pln transparan aplikasi pln mobile daftar laku...,1,2


In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(df['text_akhir'], df['polarity'], test_size=0.2, random_state=42)

# Tokenize text
tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences
max_sequence_length = 100
X_train_pad = pad_sequences(X_train_seq, maxlen=max_sequence_length)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_sequence_length)


In [ ]:
model = Sequential()
model.add(Embedding(input_dim=1000, output_dim=128, input_length=max_sequence_length))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

model.compile(optimizer='RMSprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
# Train the model
model.fit(X_train_pad, y_train, epochs=10, batch_size=64)

Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 60s 223ms/step - accuracy: 0.6390 - loss: 0.7915
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 83s 228ms/step - accuracy: 0.8991 - loss: 0.3064
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 55s 221ms/step - accuracy: 0.9246 - loss: 0.2301
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 84s 229ms/step - accuracy: 0.9338 - loss: 0.2033
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 80s 222ms/step - accuracy: 0.9484 - loss: 0.1729
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 55s 222ms/step - accuracy: 0.9491 - loss: 0.1568
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 89s 248ms/step - accuracy: 0.9532 - loss: 0.1526
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 75s 222ms/step - accuracy: 0.9548 - loss: 0.1492
Epoch 9/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 57s 229ms/step - accuracy: 0.9649 - loss: 0.1264
Epoch 10/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 81s 225ms/step - accuracy: 0.9636 - loss: 0.1383


In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test_pad, y_test)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')

125/125 ━━━━━━━━━━━━━━━━━━━━ 5s 39ms/step - accuracy: 0.9299 - loss: 0.2280
Test Loss: 0.2280
Test Accuracy: 0.9300


In [ ]:
# Save the model
filename = 'sentiment_analysis_model.pkl'
pickle.dump(model, open(filename, 'wb'))

# Save the tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Save the LabelEncoder
with open('label_encoder.pickle', 'wb') as handle:
    pickle.dump(encoder, handle, protocol=pickle.HIGHEST_PROTOCOL)

from google.colab import files
files.download('sentiment_analysis_model.pkl')
files.download('tokenizer.pickle')
files.download('label_encoder.pickle')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **Pembangunan Model Random Forest | Extraction Feature : TF-IDF**

disini saya akan menggunakan model Random Forest dengan ektrasi fitur TF-IDF, serta melakukan pembagian data 70/30

In [ ]:
df.dropna(inplace = True)

In [ ]:
X = df['text_akhir']
y = df['polarity']

tfidf = TfidfVectorizer(max_features=4000)
X_tfidf = tfidf.fit_transform(X)

# Konversi hasil ekstraksi fitur menjadi dataframe
features_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf.get_feature_names_out())
features_df

# Bagi data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.3, random_state=1)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier()
random_forest.fit(X_train.toarray(), y_train)

y_pred_train_rf = random_forest.predict(X_train.toarray())
y_pred_test_rf = random_forest.predict(X_test.toarray())

accuracy_train_rf = accuracy_score(y_pred_train_rf, y_train)
accuracy_test_rf = accuracy_score(y_pred_test_rf, y_test)

In [ ]:
# Menampilkan akurasi
print('Random Forest - accuracy_train:', accuracy_train_rf)
print('Random Forest - accuracy_test:', accuracy_test_rf)

Random Forest - accuracy_train: 0.9998571428571429
Random Forest - accuracy_test: 0.8751666666666666


# **Pembangunan Model SVM | Extraction Feature : FastText**

disini saya akan menggunakan model Random Forest dengan ektrasi fitur TF-IDF, serta melakukan pembagian data 70/30

In [ ]:
X = df['text_akhir']
y = df['polarity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

with open('train.txt', 'w', encoding='utf-8') as f:
    for text, label in zip(X_train, y_train):
        f.write(f'__label__{label} {text}\n')

model_ft = fasttext.train_supervised(input="train.txt", epoch=25, lr=1.0, wordNgrams=2, verbose=2, minCount=1)

X_train_features = [model_ft.get_sentence_vector(text) for text in X_train.fillna('').astype(str)]
X_test_features = [model_ft.get_sentence_vector(text) for text in X_test.fillna('').astype(str)]

In [ ]:
svm_model = SVC(kernel='linear')
svm_model.fit(X_train_features, y_train)

y_pred_train_svm = svm_model.predict(X_train_features)
y_pred_test_svm = svm_model.predict(X_test_features)

accuracy_train_svm = accuracy_score(y_pred_train_svm, y_train)
accuracy_test_svm = accuracy_score(y_pred_test_svm, y_test)

In [ ]:
# Menampilkan akurasi
print('SVM - accuracy_train:', accuracy_train_rf)
print('SVM - accuracy_test:', accuracy_test_rf)

SVM - accuracy_train: 0.9998571428571429
SVM - accuracy_test: 0.8751666666666666
